In [1]:
import re
import pandas as pd
import numpy as np

import os
import sys
sys.path.append(os.path.join(os.path.abspath(''), '..'))
import utils.ssvep_analysis as sa

In [2]:
file_path = '../Data/2023-12-03/Marco_Deuteranomaly_0.0_mixed-deuteranomaly-series_frequency_16.5_tile-size144_90993c02-de06-4745-8fb8-c4ecbeaebe08.csv'
df = pd.read_csv(file_path, skiprows=10, header=None)

with open(file_path, 'r') as file:
    content = file.read()
    
match = re.search(r'Frequency:\s*(\d+(\.\d+)?)', content)
stimulus_frequency = float(match.group(1)) if match else None

eeg_data = df.to_numpy()[:, 1:9].astype(float)
marker = df.to_numpy()[:, -1].flatten()

sa.update_default_stimulus_frequency(stimulus_frequency)